In [17]:
theta=1
circAngle=44.0/7
delTheta=circAngle/(r[1]-r[0]+1)
print(circAngle)
print(delTheta)
print(round(theta/delTheta))

6.28571428571
0.392857142857
3.0


In [22]:
import random
import csv
import sys
import copy
import math

class Vector :
        theta=0
        mag=0
        circAngle=44.0/7

        def __init__(self, val, r) :
                delTheta=circAngle/(r[1]-r[0]+1)
                self.mag=1
                self.theta=val*delTheta
               
        def modValue(self, r) :
                delTheta=circAngle/(r[1]-r[0]+1)
                return round(self.theta/delTheta)

        def setVector(self, val, r) :
                delTheta=circAngle/(r[1]-r[0]+1)
                self.mag=1
                self.theta=val*delTheta

        def setVector(self, val, mag, r) :
                self.mag=mag
                self.theta=val*delTheta

        def addVector(self,v) :
                R= Vector(0, [0,15])
                y=v.mag*math.sin(v.theta)+self.mag*math.sin(self.theta)
                x=v.mag*math.cos(v.theta)+self.mag*math.cos(self.theta)
                if x==0 :
                        if y>0 :
                                R.theta=(circAngle/4)
                        else :
                                R.theta=(circAngle*3/4)
                else:
                        R.theta=math.atan(abs(y/x))

                if y<0 and x>0 :
                        R.theta=circAngle-R.theta
                elif y<0 and x<0 :
                        R.theta=circAngle/2+R.theta
                elif y>0 and x<0 :
                        R.theta=circAngle/2-R.theta

                if x==0 and y==0 :
                        #insert the chance logic
                        R.theta=self.theta+circAngle/4

                R.mag=(y**2+x**2)**0.5
                return R

In [23]:
def generateCLAVector(number,size,sparseness):
    sample_vector=[1]*int(sparseness*size)+[0]*(size-int(sparseness*size))
    vector=[[1]*int(sparseness*size)+[0]*(size-int(sparseness*size))]*number
    for j in range(0,number):
        vector[j]=sample_vector[:]
        random.shuffle(vector[j])
    return vector

def sparsify(vector):
    sparse_vector=list()
    for i in range(0,len(vector)):
        if vector[i]==1:
            sparse_vector.append(i)
    return sparse_vector

def generateConversionSet(number,size,r):
    #number - number of mcr vectors = dimensions in cla vector
    #size - length of each mcr vector = dimensionality of mcr space
    conversionSet=list()
    for j in range(0,number):
        conversionSet.append(list())
        for i in range(0,size):
            conversionSet[j].append(random.randint(r[0],r[1]))
    return conversionSet

def multiplyMCR(mcr1,mcr2,r):
    mcrR=list()
    for i in range(0,len(mcr1)):
        mcrR.append((mcr1[i]+mcr2[i])%(r[1]-r[0]+1)) #does not account for ranges with negative integers 
    return mcrR

def inverseMCR(mcr,r):
    mcrR=list()
    for i in range(0,len(mcr)):
            mcrR.append((r[1]+1-mcr[i])%r[1]-r[0]+1)
    return mcrR

def sumTable(r):
        circAngle=2*(22/7)
        delTheta=circAngle/(r[1]-r[0]+1)
        table=list()
        for i in range(0,r[1]-r[0]+1):
                table.append(list())
                for j in range(0,r[1]-r[0]+1):
                    y=math.sin(i*delTheta)+math.sin(j*delTheta)
                    x=math.cos(i*delTheta)+math.cos(j*delTheta)
                    if x==0 :
                            if y>0 :
                                table[i].append(circAngle/4)
                            else :
                                table[i].append(circAngle*3/4)
                    else:
                            table[i].append(math.atan(y/x))
        return table

def vSum(a,b,r):
        R=0
        modular=r[1]-r[0]+1
        hMod=modular/2
        if a%modular>hMod and b%modular>hMod or a%modular<hMod and b%modular<hMod :
                R=(a+b)/2
        else :
                if a>b :
                    c=(modular-(a-b))/2
                    R=a+c
                else :
                    c=(modular-(b-a))/2
                    R=b+c
        return R

def vectorSum(a,table,r,isTrig):
        modular=r[1]-r[0]+1
        if isTrig==True :
            circAngle=2*(22/7)
            delTheta=circAngle/(r[1]-r[0]+1)
            add=Vector(a[0],r)
            for i in range(1,len(a)):
                v=Vector(a[i],r)
                add=add.addVector(v)
        else :
            add=a[0]
            for i in range(1,len(a)):
                add=vSum(add,a[i],r)

        return (add.modValue(r))%modular

def addMCR(mcrs,r):
    mcrR=list()
    table=sumTable(r)
    for j in range(0,len(mcrs[0])):
        mcrR.append(0)
        values=[ mcrs[i][j] for i in range(0,len(mcrs)) ]
        mcrR[j]= vectorSum(values,table,r,True)
    return mcrR


def convertCLAtoMCR(vector,conversionSet,r,indexSet):
    mcrVector=list()
    sp_vector=sparsify(vector)
    selConvSet=[ conversionSet[i] for i in sp_vector ]
    return addMCR(selConvSet,r)
    '''... This was previous implementation
        mcrVector=conversionSet[sp_vector[0]]
        for i in range(1,len(sp_vector)):
            mcrVector=addMCR(mcrVector,conversionSet[sp_vector[i]],r)
        return mcrVector
    ...'''

def distance(vector1,vector2,r):
    distance=0
    modular=r[1]-r[0]+1
    for i in range(0,len(vector1)):
        distance+=min((vector1[i]-vector2[i])%modular,(vector2[i]-vector1[i])%modular)
    return distance

def addNoise(vector,pct):
   ones_index=sparsify(vector)
   selected=random.sample(ones_index,int(pct*len(ones_index)))
   zeroes_index=[ range(0,len(vector))[i] for i in range(0,len(vector)) if i not in ones_index]
   selected_zeroes=random.sample(zeroes_index,int(pct*len(ones_index)))
   selected.extend(selected_zeroes)
   noisy=vector[:]
   for i in selected:
        noisy[i]^=1 # there are many ways to flip 1->0 and 0->1 like var=not var, var = (0,1)[var], var=1-var
   return noisy

def analyzeAddition(vectors,r,times):
        #TODO add definition
        return

In [27]:
number_of_vectors=100
nV=number_of_vectors
sparseness=0.02
noise=0.0
dimension_of_cla_vectors=1024
dimension_of_mcr_vectors=1024
r=[0,15]
mcrV=list()

In [35]:
nV=number_of_vectors

In [28]:
claV=generateCLAVector(number_of_vectors,dimension_of_cla_vectors,sparseness)
print(distance(claV[0],claV[1],[0,1]))

40


In [29]:
convSet=generateConversionSet(dimension_of_cla_vectors,dimension_of_mcr_vectors,r)
idxSet=generateConversionSet(dimension_of_cla_vectors,dimension_of_cla_vectors,r)
print (distance(convertCLAtoMCR(claV[0],convSet,r,idxSet),convertCLAtoMCR(claV[1],convSet,r,idxSet),r))

4105.0


In [30]:
for i in range(0,number_of_vectors):
    mcrV.append(convertCLAtoMCR(claV[i],convSet,r,idxSet))

In [33]:
import numpy as np

In [36]:
CLA_dist=list()
MCR_dist=list()
random_MCR_dist=list()
CLA_noisy_dist=list()
MCR_noisy_dist=list()

avg_CLA_dist=0
avg_MCR_dist=0
avg_random_MCR_dist=0
avg_CLA_noisy_dist=0
avg_MCR_noisy_dist=0

for i in range(0,number_of_vectors):
    for j in range(i+1,number_of_vectors):
        CLA_dist.append(distance(claV[i],claV[j],[0,1]))
        MCR_dist.append(distance(mcrV[i],mcrV[j],r))
        random_MCR_dist.append(distance(convSet[i],convSet[j],r))

for i in range(0,number_of_vectors):
    noisyV=addNoise(claV[i],noise)
    CLA_noisy_dist.append(distance(claV[i],noisyV,[0,1]))
    MCR_noisy_dist.append(distance(mcrV[i],convertCLAtoMCR(noisyV,convSet,r,idxSet),r))


f = open('dist_datapoints','w')
csvwriter=csv.writer(f)
csvwriter.writerow(CLA_dist)
csvwriter.writerow(MCR_dist)
csvwriter.writerow(CLA_noisy_dist)
csvwriter.writerow(MCR_noisy_dist)
f.close()

avg_CLA_dist=np.mean(CLA_dist)
avg_MCR_dist=np.mean(MCR_dist)
avg_MCR_noisy_dist=np.mean(MCR_noisy_dist)
avg_CLA_noisy_dist=np.mean(CLA_noisy_dist)
avg_random_MCR_dist=np.mean(random_MCR_dist)

print ("Vectors used="+str(nV))
print ("Average random MCR distance="+str(avg_random_MCR_dist) ) #average distance between any random MCR vectors
print ("Average CLA Distance="+str(avg_CLA_dist) )#average distance of combination of all points(CLA vectors) in CLA space
print ("Average MCR Distance="+str(avg_MCR_dist) )#average distance of combination of all points(MCR vectors) in MCR space
print ("Average CLA Distance in "+str(noise)+" noisy CLA="+str(avg_CLA_noisy_dist) )
#average distance between MCR projection of CLA vector and its noisy version
print ("Average MCR Distance from "+str(noise)+" noisy CLA="+str(avg_MCR_noisy_dist) )
#average distance between MCR projection of CLA vector and its noisy version

sdv_CLA_dist=np.std(CLA_dist)
sdv_MCR_dist=np.std(MCR_dist)
sdv_MCR_noisy_dist=np.std(MCR_noisy_dist)
sdv_CLA_noisy_dist=np.std(CLA_noisy_dist)
sdv_random_MCR_dist=np.std(random_MCR_dist)

print ("Standard Deviation random MCR Distance="+str(sdv_random_MCR_dist) )#std dev distance of random MCR vectors
print ("Standard Deviation CLA Distance="+str(sdv_CLA_dist) )#std dev distance of combijation of all points(CLA vectors) in CLA space
print ("Standard Deviation MCR Distance="+str(sdv_MCR_dist) )#std dev distance of combination of all points(MCR vectors) in MCR space
print ("Standard Deviation CLA Distance in "+str(noise)+" noisy CLA="+str(sdv_CLA_noisy_dist) )#std dev distance between MCR projection of CLA vector and its noisy version
print ("Standard Deviation MCR Distance from "+str(noise)+" noisy CLA="+str(sdv_MCR_noisy_dist)) #std dev distance between MCR projection of CLA vector and its noisy version

Vectors used=100
Average random MCR distance=4095.26646465
Average CLA Distance=39.2387878788
Average MCR Distance=4042.84323232
Average CLA Distance in 0.0 noisy CLA=0.0
Average MCR Distance from 0.0 noisy CLA=0.0
Standard Deviation random MCR Distance=74.7728135291
Standard Deviation CLA Distance=1.23023794424
Standard Deviation MCR Distance=109.264158039
Standard Deviation CLA Distance in 0.0 noisy CLA=0.0
Standard Deviation MCR Distance from 0.0 noisy CLA=0.0


In [72]:
def probe(mcrV,probe):
    return distance(mcrV,probe,[0,15])<3750

In [68]:
print(np.array(sparsify(claV[1])))
v=np.array([1,2],dtype=int)
w=np.array([3,4,5,6,7],dtype=int)
v=np.append(v,4)
v=np.append(v,5)
tp=len(np.intersect1d(w,v))
fp=len(np.setdiff1d(v,w))
fn=len(np.setdiff1d(w,v))
print(w)
print(v)
print(np.intersect1d(w,v))
print(np.setdiff1d(v,w))
print(np.setdiff1d(w,v))
print(tp)
print(fp)
print(fn)
print(np.intersect1d(w,v))

[101 113 200 275 298 319 355 406 430 461 503 516 586 607 704 757 910 946
 985 996]
[3 4 5 6 7]
[1 2 4 5]
[4 5]
[1 2]
[3 6 7]
2
2
3
[4 5]


In [73]:
precision=np.array([0.]*number_of_vectors)
recall=np.array([0.]*number_of_vectors)
for i in range(0,number_of_vectors):
    v=np.array([],dtype=int)
    for j in range(0,dimension_of_cla_vectors):
        if probe(mcrV[i],convSet[j]):
            v=np.append(v,j)
            
    tp=len(np.intersect1d(np.array(sparsify(claV[i])),v))
    fp=len(np.setdiff1d(v,np.array(sparsify(claV[i]))))
    fn=len(np.setdiff1d(np.array(sparsify(claV[i])),v))
    
    precision[i]=float(tp)/(tp+fp)
    recall[i]=float(tp)/(tp+fn)
    print(i)
print("Average Precision"+str(np.mean(precision)))
print("Average Recall"+str(np.mean(recall)))

print("Std Precision"+str(np.std(precision)))
print("Std Recall"+str(np.std(recall)))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
Average Precision1.0
Average Recall1.0
Std Precision0.0
Std Recall0.0
